PART 2 AND 3


Get Village busy times : (Since Gmaps API is paid to get the busy times, I've scraped it.)

In [92]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from collections import defaultdict

def scrape_busy_times_with_days(full_url):
    """Scrape busy times along with the time of day from Google Maps using Selenium."""
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    
    try:
        # Set timeout to allow page load
        driver.set_page_load_timeout(20)
        driver.get(full_url)

        # Wait for the relevant elements (aria-labels) to be present
        WebDriverWait(driver, 15).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div[role="img"][aria-label]'))
        )

        # Find all elements that contain the aria-label with the busy time info
        busy_time_elements = driver.find_elements(By.CSS_SELECTOR, 'div[role="img"][aria-label]')

        # Extract times and busy percentages
        busy_times_data = []
        for element in busy_time_elements:
            aria_label = element.get_attribute("aria-label")
            if "busy" in aria_label:
                # Extract time and busy percentage using split
                time_str = aria_label.split("at")[1].strip()
                busy_percentage = int(aria_label.split("%")[0].strip().split()[-1])

                # Only include times with a busy percentage greater than 0%
                if busy_percentage > 0:
                    busy_times_data.append({"time": time_str, "busy_percentage": busy_percentage})

        return busy_times_data

    except Exception as e:
        print(f"An error occurred: {e}")
        return []
    finally:
        driver.quit()

def calculate_average_busy_times(busy_times_data):
    """Calculate average busy percentages for each time slot."""
    time_slots = defaultdict(list)

    # Aggregate busy percentages by time slot
    for entry in busy_times_data:
        time_slots[entry['time']].append(entry['busy_percentage'])

    # Calculate averages
    avg_busy_times = {
        time: sum(percentages) / len(percentages) for time, percentages in time_slots.items()
    }

    # Sort results by time
    return dict(sorted(avg_busy_times.items()))

# Example: Use the full Google Maps URL
full_gmaps_url = "https://www.google.com/maps/place/Village+-+The+Soul+of+India/@40.7665603,-73.523538,17z/data=!3m1!4b1!4m6!3m5!1s0x89c281752d83843d:0x1f2a365d2207b71c!8m2!3d40.7665603!4d-73.523538!16s%2Fg%2F11thj448_5?entry=ttu&g_ep=EgoyMDI0MTIwNC4wIKXMDSoASAFQAw%3D%3D"
busy_times_data = scrape_busy_times_with_days(full_gmaps_url)

if busy_times_data:
    print("Busy Times (Filtered):")
    for entry in busy_times_data:
        print(f"Time: {entry['time']}, Busy Percentage: {entry['busy_percentage']}%")

    # Calculate and display average busy times
    avg_busy_times = calculate_average_busy_times(busy_times_data)
    print("\nBusiest Times (Averaged):")
    for time, avg_percentage in avg_busy_times.items():
        print(f"Time: {time}, Average Busy Percentage: {avg_percentage:.2f}%")
else:
    print("No busy times data found or the page could not be scraped.")


Busy Times (Filtered):
Time: 11 am., Busy Percentage: 22%
Time: 12 pm., Busy Percentage: 45%
Time: 1 pm., Busy Percentage: 63%
Time: 2 pm., Busy Percentage: 55%
Time: 5 pm., Busy Percentage: 49%
Time: 6 pm., Busy Percentage: 81%
Time: 7 pm., Busy Percentage: 89%
Time: 8 pm., Busy Percentage: 77%
Time: 9 pm., Busy Percentage: 46%
Time: 11 am., Busy Percentage: 7%
Time: 12 pm., Busy Percentage: 16%
Time: 1 pm., Busy Percentage: 26%
Time: 2 pm., Busy Percentage: 27%
Time: 5 pm., Busy Percentage: 46%
Time: 6 pm., Busy Percentage: 65%
Time: 7 pm., Busy Percentage: 81%
Time: 8 pm., Busy Percentage: 72%
Time: 9 pm., Busy Percentage: 55%
Time: 11 am., Busy Percentage: 29%
Time: 12 pm., Busy Percentage: 29%
Time: 1 pm., Busy Percentage: 31%
Time: 2 pm., Busy Percentage: 25%
Time: 5 pm., Busy Percentage: 45%
Time: 6 pm., Busy Percentage: 78%
Time: 7 pm., Busy Percentage: 100%
Time: 8 pm., Busy Percentage: 83%
Time: 9 pm., Busy Percentage: 55%
Time: 11 am., Busy Percentage: 13%
Time: 12 pm., Busy

Get current temperature & rain near Village and Display both

In [83]:
import requests

def get_weather(api_key, lat, lon):
    """Fetch weather data from OpenWeatherMap."""
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=standard"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        weather = {
            "temperature_kelvin": data["main"]["temp"],  # This will now return temperature in Kelvin
            "rain": data.get("rain", {}).get("1h", 0),  # Rainfall in last hour (mm)
            "description": data["weather"][0]["description"]
        }
        return weather
    else:
        print(f"Error fetching weather data: {response.status_code}")
        return None

# Example usage
weather_api_key = "34d771536680d741c050"  # Replace with your API key
latitude, longitude = 40.766876, -73.523663  # Village coordinates
weather = get_weather(weather_api_key, latitude, longitude)

if weather:
    # Convert temperature from Kelvin to Fahrenheit
    temperature_fahrenheit = (weather['temperature_kelvin'] - 273.15) * 9/5 + 32
    
    print(f"Weather near Village - The Soul of India:")
    print(f"Temperature: {temperature_fahrenheit:.2f}°F")  # Display temperature in Fahrenheit
    print(f"Rain: {weather['rain']} mm")
    print(f"Condition: {weather['description']}")


Weather near Village - The Soul of India:
Temperature: 32.99°F
Rain: 0 mm
Condition: overcast clouds


Here, weather data is fetched using the API.

The price is adjusted dynamically. It checks if the temperature is below 45°F or if there is rain or snow. Prices are increased by 20% if conditions are unfavorable. Additionally, prices are adjusted based on the busyness of the restaurant (high or low).

A random busyness level (either "high" or "low") is assumed based on random number generation.

The menu data is read from a CSV file using pandas, and adjustments are applied to the prices dynamically.

Error handling for file reading (menu CSV), ensuring that the script doesn’t crash if the file is not found.

In [77]:
import requests
import random
import pandas as pd

def get_weather(api_key, lat, lon):
    """Fetch weather data from OpenWeatherMap and convert temp to Fahrenheit."""
    url = f"http://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={api_key}&units=metric"
    response = requests.get(url)
    if response.status_code == 200:
        data = response.json()
        temp_kelvin = data["main"]["temp"]
        temp_fahrenheit = (temp_kelvin - 273.15) * 9/5 + 32  # Convert Kelvin to Fahrenheit
        rain = data.get("rain", {}).get("1h", 0)  # Amount of rain in the last hour
        snow = data.get("snow", {}).get("1h", 0)  # Amount of snow in the last hour
        weather_description = data["weather"][0]["description"]
        
        return temp_fahrenheit, rain, snow, weather_description
    else:
        print(f"Error fetching weather data: {response.status_code}")
        return None, None, None, None

def price_adjustment(temp_fahrenheit, rain, snow, busyness, menu_data):
    """Adjust prices based on weather conditions."""
    adjusted_prices = []
    for _, row in menu_data.iterrows():
        # Clean the price by removing the dollar sign and convert it to a numeric value
        price = row["Price"].replace('$', '')  # Remove the dollar sign
        price = pd.to_numeric(price, errors='coerce')  # Convert to numeric
        
        # If the price is not valid (NaN), skip this item
        if pd.isna(price):
            print(f"Invalid price for item: {row['Item']}, skipping.")
            continue
        
        # Adjust based on weather conditions
        if temp_fahrenheit < 45 or rain > 2 or snow > 2:  # Check weather conditions
            price *= 1.2  # Increase price by 20%
        
        # Adjust based on busyness
        if busyness == "high":  # Assume the restaurant is busier than usual
            price *= 1.1  # Increase price by 10%
        
        adjusted_prices.append(price)
    
    menu_data["Adjusted_Price"] = adjusted_prices
    return menu_data

def display_menu(menu_data):
    """Display the menu with adjusted prices (without descriptions)."""
    print("Menu and Adjusted Prices:")
    for _, row in menu_data.iterrows():
        print(f"{row['Item']}: ${row['Adjusted_Price']:.2f}")

def main():
    # Replace with your own API key
    weather_api_key = "34d771536680607fa6b03632d741c050"  # Replace with your actual API key
    latitude, longitude = 40.766876, -73.523663  # Example coordinates

    # Fetch weather data
    temp_fahrenheit, rain, snow, weather_description = get_weather(weather_api_key, latitude, longitude)
    
    if temp_fahrenheit is None:
        print("Weather data unavailable. Exiting...")
        return
    
    # Assume the restaurant is busier than usual due to weather conditions
    busyness = "high" if random.random() > 0.5 else "low"  # Randomly assume busyness

    # Load menu data from CSV file
    try:
        menu_data = pd.read_csv('village_menu.csv')  # Assuming the CSV file is in the same directory
    except FileNotFoundError:
        print("Menu file not found.")
        return

    # Adjust prices based on weather and busyness
    adjusted_menu = price_adjustment(temp_fahrenheit, rain, snow, busyness, menu_data)
    
    # Display adjusted menu (without descriptions)
    display_menu(adjusted_menu)

if __name__ == "__main__":
    main()


Menu and Adjusted Prices:
Medu Vada: $7.20
Medu Vada in Sambar Bowl: $8.39
Medu Vada in Rasam Bowl: $8.39
Combo: $7.19
Combo in Sambar Bowl: $8.39
Masala Vada: $8.94
Dahi Vada: $10.74
Batata Vada: $7.20
Mysore Bonda: $7.20
Chennai Pakkoda: $7.20
Spring Rolls: $8.94
Paneer Pakkoda: $10.74
Samosa: $7.20
Aloo Tikki: $7.20
Chilly Pakora: $7.20
Tomato Soup: $8.94
Vegetables Manchow Soup: $8.94
Rasam: $8.94
Sambar: $8.94
Mulligatawny Soup: $8.94
Garden Soup: $8.94
Katchumbar Salad: $8.94
Garden Salad: $8.94
Idly: $8.94
Masala Idly: $11.94
Mini Idly: $8.94
Ghee Fried Idly: $11.94
Podi Idly: $10.74
Chilli Idly: $11.94
Idly in Sambar Bow: $10.74
Idly Rasam Bowl: $10.74
Dami Idly: $8.94
Bhel Puri: $10.74
Samosa Chat: $10.74
Aloo Tikki Chat: $11.94
Papdee Chat: $10.74
Dahi Bhalla: $10.74
Channa Chat: $10.74
Aloo Chat: $10.74
Vadda Pav: $10.74
Pav Bhaji: $16.50
Vegetables Manchurian: $15.54
Baby Corn Chilly Manchurian: $16.50
Chilly Paneer Dry: $16.50
Gobi Manchurian Dry: $15.54
Chilli Mushroom: $